# DaCy and Sentiment
DaCy currently does not include any tools for sentiment extraction, but a couple of good tools already exists. Thus DaCy wraps these in the DaCy framework, but if you use this in a publication or similar be sure to credit the original authors.

In [3]:
import dacy
import spacy

## BertTone
--- 

BertTone is a model trained by DaNLP, actually it is two. One for classification of polarity (whether a sentence is positive, negative or neutral) and subjectivity (whether a text is subjective or not).

To read more about BertTone as well as its performance matched against other models see DaNLP's [GitHub](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md).

Here I will show a simple use case of both models. Furthermore if you wish to inspect the TransformerData to see e.g. the used wordpieces you check out the `doc._.berttone_subj_trf_data` or `doc._.berttone_pol_trf_data`

In [ ]:
nlp = spacy.blank("da")
nlp = dacy.sentiment.add_berttone_subjectivity(nlp)

In [ ]:
texts = ["Analysen viser, at økonomien bliver forfærdelig dårlig", 
         "Jeg tror alligvel, det bliver godt"]

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.subjectivity)
    print(doc._.subjectivity_prop)

In [ ]:
nlp = dacy.sentiment.add_berttone_polarity(nlp)

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.polarity)
    print(doc._.polarity_prop)

BertEmotion
---

Siliar to BertTone is a BertEmoiton is a model trained by DaNLP, actually it is also two. One for classifying whether a text is emotionally laden or not, and one for emotion classification using. The possible emotions to classify from is:

- "Glæde/Sindsro"
- "Tillid/Accept"
- "Forventning/Interrese"
- "Overasket/Målløs"
- "Vrede/Irritation"
- "Foragt/Modvilje"
- "Sorg/trist"
- "Frygt/Bekymret"

Their transformerData can be accessed using `bertemotion_laden_trf_data` for the model whether a text is emotionally laden and `bertemotion_emo_trf_data` for the model predicting emotion. Similarly to above you can always use the `*_prop` prefix to extract the probabilities of each label.

In [4]:
nlp = dacy.sentiment.add_bertemotion_laden(nlp)  # whether a text is emotionally laden
nlp = dacy.sentiment.add_bertemotion_emo(nlp)    # what emotion is portrayed

NameError: name 'add_bertemotion_laden' is not defined

In [ ]:
texts = ['bilen er flot', 'jeg ejer en rød bil og det er en god bil', 'jeg ejer en rød bil men den er gået i stykker']

docs = nlp.pipe(texts)

for doc in docs:
    if doc._.laden == "No emotion":
        print(doc._.laden)
    else:
        print(doc._.emotion)

## DaVader

---

DaVader is a Danish Sentiment model developing using [Vader](https://github.com/fnielsen/afinn) and the dictionary list of [SentiDa](https://github.com/guscode/sentida) and [AFINN](https://github.com/fnielsen/afinn). This adaption is developed by Center for Humanities Computing Aarhus and by the author of this package. It is a lexicon and rule-based sentiment analysis tool which predict sentiment valence - the degree to which a text is positive or negative - as opposed to BertTone which simply predict whether or not it is.

An additional advantage of it being rule-based is that it is transparent (the entire lexion can be found in the sentiment folder) and very fast compared to transformer-based approaches.

In [ ]:
from spacy.tokens import Doc
from dacy.sentiment import da_vader_getter

Doc.set_extension("vader_da", getter=da_vader_getter)

texts = ['bilen er flot', 'jeg ejer en rød bil og det er en god bil', 'jeg ejer en rød bil men den er gået i stykker']

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.vader_da)